In [1]:
from prevo.misc import PeriodicSensor, PeriodicTimedSensor
from prevo.misc import DummyPressureSensor, DummyTemperatureSensor, DummyElectricalSensor
from prevo.plot import OscilloGraph, NumericalGraph
%matplotlib tk

# Dummy sensors

### Raw sensors

In [2]:
pressure_sensor = DummyPressureSensor()
temperature_sensor = DummyTemperatureSensor()
electrical_sensor = DummyElectricalSensor(interval=0.1, npts=20)

In [3]:
pressure_sensor.read()

{'P1 (Pa)': 3170.4946356266923,
 'P2 (Pa)': 2338.707595559375,
 'P3 (mbar)': 17.344107429793553}

### Periodic reading of sensors

In [4]:
class PressureReading(PeriodicTimedSensor):
    """Returns individual readings of pressure at each time step"""
    
    name = 'P'
    data_types = 'P (Pa)', 'P (Pa)', 'P (mbar)'
    
    def _read_sensor(self):
        return tuple(pressure_sensor.read().values())
    

class TemperatureReading(PeriodicTimedSensor):
    """Returns individual readings of temperature at each time step"""
    
    name = 'T'
    data_types = 'T (°C)', 'T (°C)'
    
    def _read_sensor(self):
        return tuple(temperature_sensor.read().values())
   

class ElectricalReading(PeriodicSensor):
    """Returns arrays of readings corresponding to different times"""
    
    name = 'V'
    data_types = 'voltage (V)', 'voltage (V)'
    
    def _read(self):
        data = electrical_sensor.read()
        return {'time (unix)': data[0],
                'values': data[1:],
                'name': self.name}


pressure_reading = PressureReading(interval=0.02)
temperature_reading = TemperatureReading(interval=0.7)
electrical_reading = ElectricalReading(interval=0.1)

readings = pressure_reading, temperature_reading, electrical_reading

### Misc. sensor config

In [5]:
data_types = {reading.name: reading.data_types for reading in readings}

data_ranges = {'P (Pa)': (1000, 4000),
               'P (mbar)': (10, 40),
               'T (°C)': (20, 30),
               'voltage (V)': (-0.5, 1.5)}

### Start periodic sensor reading

In [6]:
for reading in readings:
    reading.start()

In [7]:
pressure_reading.queue.get()

{'time (unix)': 1691764352.3738387,
 'dt (s)': 7.374997949227691e-05,
 'values': (3170.46571492631, 2338.8189327401606, 17.426214771756456),
 'name': 'P'}

# Live Graphs of sensors

In [8]:
colors = {'fig': 'lightgrey',   # set color of figure background around axes
          'T': ('k', 'b')}      # set color of curves of the two channels of sensor T

legends = {'P': ('Inlet', 'Outlet', 'Chamber')}

linestyles = {'T': ('-', '--')}

## Regular numerical plot

**ATTENTION**: one may need to restart the kernel after running one of the examples below, in order to run another example (this tends to happen with QT5, but seems to not be the case when using TK as matplotlib backends)

### Data as single time values

In [9]:
numplot = NumericalGraph(names=('P', 'T'),
                         data_types=data_types,
                         legends=legends,
                         linestyles=linestyles,
                         linestyle='.-',
                         colors=colors)

numplot.run(q_plot={'P': pressure_reading.queue,
                    'T': temperature_reading.queue},
            dt_graph=0.2)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


### Data as arrays

In [10]:
numplot = NumericalGraph(names=('V',),
                         data_types=data_types,
                         data_as_array=True)

numplot.run(q_plot={'V': electrical_reading.queue}, dt_graph=0.2)

### Mix of single times and arrays

In [11]:
numplot = NumericalGraph(names=('P', 'V'),
                         data_types=data_types,
                         data_as_array={'P': False, 'V': True},
                         time_conversion='pandas')

numplot.run(q_plot={'P': pressure_reading.queue,
                    'V': electrical_reading.queue},
            dt_graph=0.2)

## Oscilloscope-like graph

### Data as single time values

In [12]:
oscillo = OscilloGraph(names=('P', 'T'),
                       data_types=data_types,
                       data_ranges=data_ranges,
                       window_width=10,
                       colors=colors,
                       legends=legends,
                       linestyles=linestyles)

oscillo.run(q_plot={'P': pressure_reading.queue,
                    'T': temperature_reading.queue},
            dt_graph=0.02)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


### Data as arrays

In [13]:
oscillo = OscilloGraph(names=('V',),
                       data_types=data_types,
                       data_ranges=data_ranges,
                       window_width=5.7,
                       data_as_array=True)

oscillo.run(q_plot={'V': electrical_reading.queue}, dt_graph=0.02)

### Mix of single times and arrays

In [14]:
oscillo = OscilloGraph(names=('P', 'T', 'V'),
                       data_types=data_types,
                       data_ranges=data_ranges,
                       window_width=5.7,
                       colors=colors,
                       data_as_array={'P': False, 'T': False, 'V': True})

oscillo.run(q_plot={'P': pressure_reading.queue,
                    'T': temperature_reading.queue,
                    'V': electrical_reading.queue},
            dt_graph=0.02)

# Stop dummy sensors

In [15]:
for reading in readings:
    reading.stop()

Non-blocking run of P stopped.
Non-blocking run of T stopped.
Non-blocking run of V stopped.
